<a href="https://colab.research.google.com/github/drew-walkerr/Diss_Detecting_Provider_Bias/blob/main/stigmatizing_labels_regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import os
import re
import sys
import scipy
import Cython
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en import English
import numpy as np

from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive/')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/gdrive/


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


In [3]:

%cd /content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Stigmatizing Labels/1_Data Prep
%ls

/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Stigmatizing Labels/1_Data Prep
1_stigmatizing_labels_word_embeddings.py
stigma_expanded_misspellings.csv
stigmatizing_labels_dataframe_annotate.csv
stigmatizing_labels_descriptors_lexicon_stem_and_similar_round1.csv
stigmatizing_labels_lexicon_dev.ipynb
stigmatizing_labels_regex.ipynb
stigmatizing_word_list_round_1_dw_pruned.csv
stigma_word_list_round_2.csv
stigma_word_list_round_3_gpt.csv
trig-vectors-phrase.bin
trig-vectors-phrase.txt


BigQuery

In [ ]:
#Replace 'project_id' with your BigQuery project ID

from google.cloud import bigquery
client = bigquery.Client(project='detecting-bias-mimic-iii')

icd_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.diagnoses_icd
                LIMIT 1000''')

icd = client.query(icd_query).to_dataframe()



ICD codes

In [ ]:


#Filter for ICD codes for 282.60-282.69, referring to sickle cell types w/wo crisis
#2824 for thalassemia w + w/o crisis (282.41-282.42)
#SCD: 2826,2824
#Chronic Pain 3382
#Opioid dependencies: 3040,3047 (combo),
#HIV/AIDS ^042$

icds_of_interest = icd[icd['ICD9_CODE'].str.contains('2826|2824|3040|3047|3382|^042$', na=False)]
# Issues here-- will cause more leading numbers
print(icds_of_interest.head())

icds_of_interest.info()

#patients_unique = icds_of_interest['SUBJECT_ID'].drop_duplicates()


     ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
784  176454       15872   139593        3     73382
889  196755       17664   123887        7     33829
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 784 to 889
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ROW_ID      2 non-null      Int64 
 1   SUBJECT_ID  2 non-null      Int64 
 2   HADM_ID     2 non-null      Int64 
 3   SEQ_NUM     2 non-null      Int64 
 4   ICD9_CODE   2 non-null      object
dtypes: Int64(4), object(1)
memory usage: 104.0+ bytes


Notes and patients load

In [ ]:
note_query = ('''SELECT *
                FROM physionet-data.mimiciii_notes.noteevents''')
                #LIMIT 100000''')

NOTES = client.query(note_query).to_dataframe()

peek_notes = NOTES.head()
print(peek_notes)

## Patients

patient_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.patients''')

PATIENTS = client.query(patient_query).to_dataframe()

peek_patients = PATIENTS.head()
print(peek_patients)

biased_notes_patients_corpus = NOTES.merge(PATIENTS, on = 'SUBJECT_ID')

biased_notes_patients_corpus.info()
# Find unique chart types and decide to limit them
    # Remove Radiology, ECG, Respiratory, Echo notes
biased_notes_patients_corpus_filtered = biased_notes_patients_corpus[biased_notes_patients_corpus["CATEGORY"].str.contains("Radiology|ECG|Respiratory|Echo")==False]
# biased_notes_patients_corpus_filtered.to_csv("biased_notes_patients_corpus_filtered.csv")



    ROW_ID  SUBJECT_ID  HADM_ID   CHARTDATE           CHARTTIME  \
0   183661       11861   110532  2125-09-22                 NaT   
1   256904       58417     <NA>  2187-03-12                 NaT   
2   312692       11085     <NA>  2157-01-18                 NaT   
3   120578         977   128363  2176-01-21                 NaT   
4  2016292       25247   135854  2154-03-25 2154-03-25 18:50:00   

            STORETIME       CATEGORY DESCRIPTION   CGID  ISERROR  \
0                 NaT            ECG      Report   <NA>     <NA>   
1                 NaT            ECG      Report   <NA>     <NA>   
2                 NaT            ECG      Report   <NA>     <NA>   
3                 NaT            ECG      Report   <NA>     <NA>   
4 2154-03-25 18:50:00  Nursing/other      Report  18433     <NA>   

                                                TEXT  
0  Sinus tachycardia. Compared to the previous tr...  
1  Sinus bradycardia.  Since the previous tracing...  
2  Sinus tachycardia.  

Explode by sentence, regex quote match

In [ ]:
full_dataframe = biased_notes_patients_corpus_filtered
# Tokenize by sentence
nlp = English()  # just the language with no model

nlp.add_pipe('sentencizer')
full_dataframe["Sentence"] = full_dataframe["TEXT"].apply(lambda x: [sent.text for sent in nlp(x).sents])
full_dataframe = full_dataframe.explode("Sentence", ignore_index=True)
full_dataframe.rename(columns={"Unnamed: 0": "ROW_ID_new"}, inplace=True)
full_dataframe.index.name = "Sentence ID"

full_dataframe['Sentence'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)

full_dataframe.to_csv("/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/mimic_iii_sentence_tokenized.csv")




<ipython-input-7-639a220d647f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_dataframe["Sentence"] = full_dataframe["TEXT"].apply(lambda x: [sent.text for sent in nlp(x).sents])


OSError: ignored

# Regex matching all stemwords + any semantically similar word identified

In [4]:
# Read in sentence tokenized MIMIC-III

full_dataframe = pd.read_csv("/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/mimic_iii_sentence_tokenized.csv")

<ipython-input-4-ce85557fbcaa>:3: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  full_dataframe = pd.read_csv("/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/mimic_iii_sentence_tokenized.csv")


In [5]:


doubt_word_list = pd.read_csv("stigma_word_list_round_3_gpt.csv")


# Create regex which is each stem word + all similar_words identified in the word embeddings step, which will be used to search the charts
def group_and_create_regex(dataframe):
    grouped_data = dataframe.groupby('stem_word')['similar_word'].apply(list).reset_index()
    grouped_data['regex'] = grouped_data.apply(lambda row: '|'.join([re.escape(row['stem_word'])] + [re.escape(word) for word in row['similar_word']]), axis=1)
    return grouped_data

grouped_data = group_and_create_regex(doubt_word_list)
print(grouped_data)

# Create function that takes regex from each row and searches the charts

def identify_matching_strings(full_dataframe, grouped_data):
    matching_dataframes = []

    for index, row in grouped_data.iterrows():
        regex = row['regex']
        terms = row['similar_word']
        for term in terms:
            term = re.escape(term)
            matches = full_dataframe.loc[full_dataframe['Sentence'].str.contains(term, flags=re.IGNORECASE, na=False)].copy()
            matches['regex'] = regex
            matches['matched_term'] = term  # Add the matched term as a new column
            matching_dataframes.append(matches)

    if matching_dataframes:
        return pd.concat(matching_dataframes, axis=0)
    else:
        return pd.DataFrame()



result = identify_matching_strings(full_dataframe, grouped_data)
result




         stem_word                                       similar_word  \
0           abuser  [ivdabuser, abuser's, abused-abuser, abusers, ...   
1        alcoholic  [non-alcoholic/alcoholic, prealcoholic, ex-alc...   
2            angry                 [angry/disgusted, angry-disgusted]   
3         chat_gpt  [ "hysterical,  aggressive,  drug addict,  non...   
4        combative                                      [combatively]   
5        defensive  [overdefensive, counterdefensive, defensive/of...   
6     drug-seeking  [''drug-seeking, drug-seeking/drug-taking, 'dr...   
7            drunk    [drunks, drunken, drunkest, drunker, drunkenly]   
8   frequent-flyer  [frequent-flyers, frequent-flier, frequent-fvl...   
9       hysterical  [hysteric, hystericals, hysterics, schizo-hyst...   
10          junkie                       [junkies, 'junkie, junkie's]   
11     nonadherent  [non-adherent, nonadherents, onadherent, nonad...   
12    noncompliant  [non-compliant, noncompliants, 

,Sentence ID,ROW_ID_x,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,...,ROW_ID_y,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,Sentence,regex,matched_term
12929593,12929593,48563,13855,107256.0,2137-05-06,NaN,NaN,Discharge summary,Report,NaN,...,13101,M,2077-10-14,NaN,NaN,NaN,0,"Per OMR, reports that his parents were substan...",abuser|ivdabuser|abuser's|abused\-abuser|abuse...,abusers
12929712,12929712,371212,13855,107256.0,2137-05-01,2137-05-01 10:49:00,2137-05-01 10:49:22,Physician,Physician Resident Admission Note,14056.0,...,13101,M,2077-10-14,NaN,NaN,NaN,0,"Per OMR, reports that his parents were substan...",abuser|ivdabuser|abuser's|abused\-abuser|abuse...,abusers
12929975,12929975,48561,13855,125990.0,2135-07-06,NaN,NaN,Discharge summary,Report,NaN,...,13101,M,2077-10-14,NaN,NaN,NaN,0,Reports that his parents were substance abuse...,abuser|ivdabuser|abuser's|abused\-abuser|abuse...,abusers
12930089,12930089,48562,13855,108743.0,2135-12-19,NaN,NaN,Discharge summary,Report,NaN,...,13101,M,2077-10-14,NaN,NaN,NaN,0,"Per OMR, reports that his parents were substan...",abuser|ivdabuser|abuser's|abused\-abuser|abuse...,abusers
5774027,5774027,22009,5760,169133.0,2109-01-22,NaN,NaN,Discharge summary,Report,NaN,...,5453,F,2056-10-18,NaN,NaN,NaN,0,Her hepatic workup was consistent with non-alc...,alcoholic|non\-alcoholic/alcoholic|prealcoholi...,non\-alcoholic/alcoholic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18023216,18023216,1313078,3895,151195.0,2140-04-03,2140-04-03 06:18:00,2140-04-03 06:46:00,Nursing/other,Report,14518.0,...,3680,F,2066-06-07,2146-04-01,2146-04-01,NaN,1,"Lungs diminished at bases, clear bil upper lo...",reluctant|reluctantly|reluctanly|reluctants|mo...,unwilling
18060104,18060104,584615,58757,130745.0,2140-06-15,2140-06-15 22:15:00,2140-06-15 22:15:36,Physician,CCU Admission Note,20420.0,...,35766,M,2079-08-15,2143-11-06,2143-11-06,NaN,1,# STROKE - s/p R sided CVA in '[**33**] with ...,reluctant|reluctantly|reluctanly|reluctants|mo...,unwilling
18061939,18061939,52008,58757,130745.0,2140-06-19,NaN,NaN,Discharge summary,Report,NaN,...,35766,M,2079-08-15,2143-11-06,2143-11-06,NaN,1,# STROKE - s/p R sided CVA in '[**33**] with ...,reluctant|reluctantly|reluctanly|reluctants|mo...,unwilling
18062226,18062226,42197,67418,161948.0,2189-10-12,NaN,NaN,Discharge summary,Report,NaN,...,38049,M,2121-06-22,NaN,NaN,NaN,0,"He had a full work up at that time, according ...",reluctant|reluctantly|reluctanly|reluctants|mo...,unwilling


In [6]:

quoted_dataframe = result

quoted_dataframe["stigmatizing_patient_label"] = ""
quoted_dataframe["annotator_comments"] = ""
# X matching sentences from total of 9,118,501 sentences from 366,002 charts
# input code showing length of quoted_dataframe vs quoted_dataframe2
quoted_dataframe2 = quoted_dataframe.drop_duplicates(subset=['Sentence'])
quoted_dataframe2.to_csv("stigmatizing_labels_dataframe_annotate.csv")



Explore the Data


In [ ]:
from google.colab import runtime
runtime.unassign()